In [1]:
%reload_ext autoreload
%autoreload 2

import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from registration_utilities import *

from skimage.measure import mesh_surface_area, marching_cubes, correct_mesh_orientation

from itertools import izip
import pandas as pd

import time

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

Setting environment for Local Macbook Pro


In [2]:
with open('colors.txt', 'r') as f:
    colors = {l[0]: np.r_[float(l[1]), float(l[2]), float(l[3])] for l in map(lambda x: x.split(), f.readlines())}

In [3]:
atlas_volume = bp.unpack_ndarray_file(volume_dir + '/atlasVolume_icp.bp').astype(np.int8)

available_labels_sided = [labels_sided[i-1] for i in np.unique(atlas_volume) if i > 0]
available_labels_unsided = set([labelMap_sidedToUnsided[name] for name in available_labels_sided ])

IOError: [Errno 2] No such file or directory: '/home/yuncong/CSHL_volumes2//atlasVolume_icp.bp'

In [ ]:
atlasVolume_landmark_bboxes = {name: bbox_3d(atlas_volume == labels_sided_indices[name])
                           for name in available_labels_sided}

In [ ]:
atlas_mesh_dir = create_if_not_exists(os.path.join(mesh_rootdir, 'atlas'))

for name_s in available_labels_sided:

    print name_s

    xmin, xmax, ymin, ymax, zmin, zmax = atlasVolume_landmark_bboxes[name_s]

    vol = (atlas_volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == labels_sided_indices[name_s]).astype(np.float)
    
    t = time.time()

    polydata = volume_to_polydata(vol, origin=(xmin,ymin,zmin), num_simplify_iter=3, smooth=True)

    sys.stderr.write('time: %.2f\n' % (time.time() - t) ) # ~ 150s for outerContour; 93s for sp5
    
    save_mesh_stl(polydata, atlas_mesh_dir + "/atlasVolume_%(name)s_smoothed.stl" % {'name': name_s})

In [18]:
# Load Atlas meshes

atlas_polydata_list = {}

for name_s in available_labels_sided:

    fn = mesh_rootdir + "/atlas/atlasVolume_%(name)s_smoothed.stl" % {'name': name_s}

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()    
    atlas_polydata_list[name_s] = polydata

In [ ]:
# Load subject meshes

In [90]:
subject_localAdjusted_polydata_list_allStacks = {}

In [95]:
# stack = 'MD593'

# for stack in ['MD589', 'MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602', 'MD603']:
# for stack in ['MD589', 'MD594', 'MD585']:

for stack in ['MD594']:

    atlasAlignParams_dir = '/home/yuncong/CSHL_atlasAlignParams_atlas/'
    with open(atlasAlignParams_dir + '/%(stack)s/%(stack)s_3dAlignParams.txt' % {'stack': stack}, 'r') as f:
        lines = f.readlines()
        global_params = np.array(map(float, lines[0].strip().split()))
    #     atlas_xdim, atlas_ydim, atlas_zdim  = np.array(map(float, lines[1].strip().split()))
        atlas_centroid = np.array(map(float, lines[2].strip().split()))
        test_xdim, test_ydim, test_zdim = np.array(map(int, lines[3].strip().split()))
        test_centroid = np.array(map(float, lines[4].strip().split()))


    parameters_allLandmarks = {}
    atlas_centroid_allLandmarks = {}
    test_centroid_allLandmarks = {}

    for name in available_labels_sided:

        with open(atlasAlignParams_dir + '/%(stack)s/%(stack)s_%(name)s_transformUponAffineProjection.txt' % \
                            {'stack': stack, 'name': name}, 'r') as f:
            lines = f.readlines()
            params = np.array(map(float
                                  , lines[0].strip().split()))
            test_xdim, test_ydim, test_zdim = np.array(map(int, lines[1].strip().split()))
            atlas_c = np.array(map(float, lines[2].strip().split()))
            test_c = np.array(map(float, lines[3].strip().split()))

        parameters_allLandmarks[name] = params
        atlas_centroid_allLandmarks[name] = atlas_c
        test_centroid_allLandmarks[name] = test_c


    subject_localAdjusted_polydata_list = {}

    for name_s in available_labels_sided:

        fn = mesh_rootdir + "/%(stack)s/%(stack)s_localAdjustedVolume_%(label)s_smoothed.stl" % {'stack':stack, 'label':name_s}

        if os.path.exists(fn):
            vertices, faces = load_mesh_stl(fn)

    #         vertices = transform_points_inverse(global_params, pts_prime=vertices, 
    #                                             c_prime=test_centroid_allLandmarks[name], 
    #                                             c=atlas_centroid_allLandmarks[name])

            vertices = transform_points_inverse(global_params, pts_prime=vertices, 
                                        c_prime=test_centroid, c=atlas_centroid)

            polydata = mesh_to_polydata(vertices, faces)

            subject_localAdjusted_polydata_list[name_s] = polydata
            
    subject_localAdjusted_polydata_list_allStacks[stack] = subject_localAdjusted_polydata_list

fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.01 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 s

In [ ]:
# subject_atlasProjected_polydata_list = {}

# for name_s in available_labels_sided:

#     fn = mesh_rootdir + "/%(stack)s/%(stack)s_atlasProjectedVolume_%(label)s_smoothed.stl" % {'stack':stack, 'label':name_s}

#     if os.path.exists(fn):
#         vertices, faces = load_mesh_stl(fn)
#         vertices = transform_points_inverse(global_params, pts_prime=vertices, 
#                                             c_prime=test_centroid, c=atlas_centroid)
#         polydata = mesh_to_polydata(vertices, faces)
        
#         subject_atlasProjected_polydata_list[name_s] = polydata

In [6]:
stack = 'MD594'

In [7]:
from registration_utilities import *

# load in annotation space
vertices, faces = load_mesh_stl(mesh_rootdir + "/%(stack)s/%(stack)s_%(name)s_gaussianSmoothed.stl" % \
                                {'stack': stack, 'name': 'outerContour'})

ann_xmin, ann_xmax, ann_ymin, ann_ymax, ann_zmin, ann_zmax = \
np.loadtxt(volume_dir + '%(stack)s/volume_%(stack)s_annotation_withOuterContour_limits.txt' % {'stack': stack}, dtype=np.int)

sco_xmin, sco_xmax, sco_ymin, sco_ymax, sco_zmin, sco_zmax = \
np.loadtxt(volume_dir + '%(stack)s/%(stack)s_scoreVolume_limits.txt' % {'stack': stack}, dtype=np.int)

# convert to score volume space
vertices_alignedToScoreVolume = vertices + (ann_xmin,ann_ymin,ann_zmin) - (sco_xmin,sco_ymin,sco_zmin)

polydata_outerContour = mesh_to_polydata(vertices_alignedToScoreVolume, faces)

contour_mapper = vtk.vtkPolyDataMapper()
contour_mapper.SetInputData(polydata_outerContour)

contour_actor = vtk.vtkActor()
contour_actor.SetMapper(contour_mapper)
# contour_actor.GetProperty().SetRepresentationToWireframe()
contour_actor.GetProperty().SetColor(1,1,1)
contour_actor.GetProperty().SetOpacity(.2)

NameError: name 'mesh_rootdir' is not defined

In [4]:
import randomcolor

def random_colors(count):
    rand_color = randomcolor.RandomColor()
    random_colors = [map(int, rgb_str[4:-1].replace(',', ' ').split()) 
                     for rgb_str in rand_color.generate(luminosity="bright", count=count, format_='rgb')]
    return random_colors

In [5]:
rand_colors = np.array(random_colors(len(subject_localAdjusted_polydata_list_allStacks)))

NameError: name 'subject_localAdjusted_polydata_list_allStacks' is not defined

In [118]:
all_actors = []

atlas_mesh_actors = [actor_mesh(polydata, color=colors[labelMap_sidedToUnsided[name_s]], opacity=1., wireframe=False, ) 
               for name_s, polydata in atlas_polydata_list.iteritems()]

all_actors = [contour_actor] + atlas_mesh_actors

# launch_vtk(all_actors, init_angle='45', interactive=False, snapshot_fn='/home/yuncong/atlas_colored.png')
launch_vtk(all_actors, init_angle='45', interactive=True)
# launch_vtk(all_actors, init_angle='coronal', interactive=True)

In [123]:
all_actors = []

# atlas_mesh_actors = [actor_mesh(polydata, (1,1,1), opacity=.5, wireframe=True) 
#                for name_s, polydata in atlas_polydata_list.iteritems() if labelMap_sidedToUnsided[name_s] == '7N']

# all_actors += atlas_mesh_actors

for stack_ind, (stack, polydata_list) in enumerate(subject_localAdjusted_polydata_list_allStacks.items()[:3]):

#     subject_mesh_actors = [actor_mesh(polydata, colors[labelMap_sidedToUnsided[name_s]], opacity=1., wireframe=True) 
#                    for name_s, polydata in polydata_list.iteritems()]

#     c = np.random.randint(0, 255, (3, )) / 255.

    c = rand_colors[stack_ind] / 255.
    
#     subject_mesh_actors = [actor_mesh(polydata, c, opacity=1., wireframe=True) 
#                    for name_s, polydata in polydata_list.iteritems() if labelMap_sidedToUnsided[name_s] == '7N']

    subject_mesh_actors = [actor_mesh(polydata, c, opacity=1., wireframe=False) 
                   for name_s, polydata in polydata_list.iteritems()]

    all_actors += subject_mesh_actors
    
all_actors += [contour_actor]
    
launch_vtk(all_actors, init_angle='45', interactive=False, snapshot_fn='/home/yuncong/brains_3.png')

In [105]:
all_actors = []

atlas_mesh_actors = [actor_mesh(polydata, colors[labelMap_sidedToUnsided[name_s]], opacity=.5) 
               for name_s, polydata in atlas_polydata_list.iteritems()]

# all_actors += atlas_mesh_actors

for stack_ind, (stack, polydata_list) in enumerate(subject_localAdjusted_polydata_list_allStacks.iteritems()):

#     subject_mesh_actors = [actor_mesh(polydata, colors[labelMap_sidedToUnsided[name_s]], opacity=1., wireframe=True) 
#                    for name_s, polydata in polydata_list.iteritems()]

#     c = np.random.randint(0, 255, (3, )) / 255.
    c = np.array(random_colors(1)[0]) / 255.
    subject_mesh_actors = [actor_mesh(polydata, c, opacity=1., wireframe=True) 
                   for name_s, polydata in polydata_list.iteritems()]

    all_actors += subject_mesh_actors
    
all_actors += [contour_actor]
    
launch_vtk(all_actors, init_angle='45')